# Network building

Based on artists sharing a common announcement/exhibition: using *announcements.json* / *announcements_contemporary.json* 

## Initial steps

In [1]:
import json
with open('data/artists_cleaned_v1.txt', 'r', encoding='utf-8') as f:
    artists = f.read().splitlines()

with open('data/announcements.json', 'r', encoding='utf-8') as f:
    announcements = json.load(f)

## 1) Network building

Something like:

- Add all artists (from the txt file) to a node in the graph
- Run through all announcements, add edges between artists that share an announcement

- Graph should be undirected + weighted, weight = number of announcements shared

In [2]:
#import graph_tool as gt
#from graph_tool import inference
from graph_tool.all import * #Otherwise, draw_hierarchy will not be found in any case (even if importing graph_tool.draw)

In [ ]:
g = Graph(directed=False)

# Create a dictionary to map artist names to vertices
artist_to_vertex = {}

for announcement in announcements.values():
    announcement_artists = announcement['artists']
    for artist in artists:
        # If the artist is not already in the graph, add them
        if artist not in artist_to_vertex:
            v = g.add_vertex()
            artist_to_vertex[artist] = v

Make an artist name from vertex dict(/flip out artist_to_vertex dictionary) so we can look up artists by vertex's property 

In [ ]:
artist_name = g.new_vertex_property("string") #vertex_to_artist = {v: artist for artist, v in artist_to_vertex.items()}
for v, artist in artist_to_vertex.items():
    artist_name[v] = artist

g.vertex_properties["artist_name"] = artist_name

Add edges:

In [ ]:
import itertools

edge_weights = {}
for announcement in announcements.values():
    artists_ = announcement['artists']
    for artist1, artist2 in itertools.combinations(artists_, 2):
        # Check if the edge already exists (this returns the edge, or None)
        edge = g.edge(artist_to_vertex[artist1], artist_to_vertex[artist2])
        if edge:
            edge_weights[edge] += 1
        else:
            edge = g.add_edge(artist_to_vertex[artist1], artist_to_vertex[artist2])
            edge_weights[edge] = 1

Add edge weight property to the edges

In [ ]:
weight = g.new_edge_property("int")
for edge, weight_value in edge_weights.items():
    weight[edge] = weight_value
g.edge_properties["weight"] = weight

In [ ]:
#Print amount of vertices and edges
print(g.num_vertices())
print(g.num_edges())

This is a bit too much. The nested block model computation crashes, so we have to compute for a smaller subset of the data.<br>
(I ran this locally, possibly Jupyter notebook can be faster)

In [6]:
def create_subgraph_from_names(g, names, name_to_vertex):
    # Create a new graph
    g_sub = Graph(directed=False)

    # Create a dictionary to map names to new vertices
    name_to_new_vertex = {}

    # Create a vertex property map for the names
    name_property = g_sub.new_vertex_property("string")

    # Add vertices to the new graph
    for name in names:
        if name in name_to_vertex:
            v = g_sub.add_vertex()
            name_property[v] = name
            name_to_new_vertex[name] = v  # Map the name to the new vertex

    g_sub.vertex_properties["name"] = name_property

    # Create an edge property map for the weights
    edge_weights = g_sub.new_edge_property("int")

    for name1, name2 in itertools.combinations(names, 2):
        # Check if the edge already exists in the original graph
        if name1 in name_to_vertex and name2 in name_to_vertex:
            edge = g.edge(name_to_vertex[name1], name_to_vertex[name2])
            if edge:
                # If the edge exists, copy its weight to the subgraph
                edge_sub = g_sub.add_edge(name_to_new_vertex[name1], name_to_new_vertex[name2])
                edge_weights[edge_sub] = g.edge_properties["weight"][edge]

    g_sub.edge_properties["weight"] = edge_weights

    return g_sub

def create_subgraph_from_edges(g, edges):
    # Create a new graph
    g_sub = Graph(directed=False)

    # Create a dictionary to map names to new vertices
    name_to_new_vertex = {}

    # Create a vertex property map for the names
    name_property = g_sub.new_vertex_property("string")

    # Add vertices to the new graph
    for edge in edges:
        name1 = g.vertex_properties["artist_name"][edge.source()]
        name2 = g.vertex_properties["artist_name"][edge.target()]
        if name1 not in name_to_new_vertex:
            v1 = g_sub.add_vertex()
            name_property[v1] = name1
            name_to_new_vertex[name1] = v1  # Map the name to the new vertex
        if name2 not in name_to_new_vertex:
            v2 = g_sub.add_vertex()
            name_property[v2] = name2
            name_to_new_vertex[name2] = v2  # Map the name to the new vertex

    g_sub.vertex_properties["name"] = name_property

    # Create an edge property map for the weights
    edge_weights = g_sub.new_edge_property("int")

    for edge in edges:
        name1 = g.vertex_properties["artist_name"][edge.source()]
        name2 = g.vertex_properties["artist_name"][edge.target()]
        edge_sub = g_sub.add_edge(name_to_new_vertex[name1], name_to_new_vertex[name2])
        edge_weights[edge_sub] = g.edge_properties["weight"][edge]

    g_sub.edge_properties["weight"] = edge_weights

    return g_sub

We construct a networks: `g_double` which only has edges between artists that share 2+ announcements

In [7]:
#Construct g_doubles: a subgraph of g with only edges with weight > 1, using create_subgraph_from_names
g_doubles = create_subgraph_from_edges(g, [edge for edge in g.edges() if g.edge_properties["weight"][edge] > 1])

NameError: name 'g' is not defined

In [ ]:
state = minimize_nested_blockmodel_dl(g_double)
state.draw(output="block_model_doubles.png")

The other network is even smaller. It is the network of those artists, that appear in the *PainterPalette* dataset (there is typically a lot of information about these artists). Roughly ~1000

In [3]:
import pandas as pd

url = "https://raw.githubusercontent.com/me9hanics/PainterPalette/main/PainterPalette.csv"
artists = pd.read_csv(url)

In [ ]:
#Find the intersection of the artists in the graph and the artists in the dataset
artists_in_graph = set(artist_name[v] for v in g.vertices())
artists_in_dataset = set(artists['artist'])
artists_in_both = artists_in_graph.intersection(artists_in_dataset)

print(len(artists_in_both))

In [4]:
g_selected = create_subgraph_from_names(g, artists_in_both, artist_to_vertex)
print(g_selected.num_edges())

NameError: name 'g' is not defined

Let's fit a blockmodel on them to see if we can find some things in common in communities:

In [ ]:
state = minimize_nested_blockmodel_dl(g_selected)
state.draw(output="images/artists_selected_block_model.png") #SVG is nicer, but this would take too long to render